In [1]:
import json
from src import preparedata

In [2]:
params=json.load(open('params/params.json','r'))

In [3]:
dataset=preparedata.readGPDData(params['dataprepinargs'])

In [4]:
dataset.preparedata()

  0%|          | 0/31 [00:00<?, ?it/s]